In [12]:
import os

os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-24"
print(os.environ.get("JAVA_HOME"))

os.environ["OPENAI_API_KEY"] = "sk-socialthingum-Lzt6aXn0b7k2ia0tjcmqT3BlbkFJLWlJZwLsUuKQ6OSfzwgC"

C:\Program Files\Java\jdk-24


In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import matplotlib.pyplot as plt
import pandas as pd
import pyterrier as pt  

In [14]:
if not pt.started():
    pt.init()

C:\Users\figio\AppData\Local\Temp\ipykernel_17612\3057724015.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


## Setup

In [15]:
df = pd.read_csv("C:\\Users\\figio\Desktop\\tryKaggle\\archive\\sampled_dataset.csv")
print(df.columns)
df = df.rename(columns={'Unnamed: 0': 'id'})
print(df.head())

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\figio\AppData\Local\Temp\ipykernel_17612\3405669283.py:1: SyntaxWarning: invalid escape sequence '\D'
  df = pd.read_csv("C:\\Users\\figio\Desktop\\tryKaggle\\archive\\sampled_dataset.csv")


Index(['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source',
       'NER', 'dir_word_count', 'ner_len'],
      dtype='object')
        id                         title  \
0  2015528  Marinated Flank Steak Recipe   
1  1608734           French Chicken Stew   
2   778500                Glazed Carrots   
3  1334975               Moms Pie Dough    
4   116562      Pretzel Salad Or Dessert   

                                         ingredients  \
0  ["1 1/2 pound flank steak", "1/2 c. finely min...   
1  ["1 tablespoon rosemary", "1 teaspoon thyme", ...   
2  ["3 to 4 carrots", "1 1/2 Tbsp. butter", "1/3 ...   
3  ["4.5 Cups Flour", "1.5 Tsp Salt", "Pinch Baki...   
4  ["2 c. crushed small thin pretzels (sticks)", ...   

                                          directions  \
0  ['Remove tenderloin from steak.', 'Score meat....   
1  ["combine all ingredients in slow cooker (6 qu...   
2  ['Cook 3 to 4 carrots; cut crosswise in 1-inch...   
3  ['Mix all dry ingredients in

In [16]:
# Percorso assoluto all'indice
index_path = r"C:\\Users\\figio\\Desktop\\cooking-mama\\Riccardo Finigi\\index_fields"

# Verifica se l'indice esiste già
if os.path.exists(index_path) and os.path.exists(os.path.join(index_path, "data.properties")):
    print("✔️ Index already exists. Using existing index.")
    indexref = index_path
else:
    print("🛠️ Index not found. Building new index...")

    documents_fields = [
        {
            'docno': str(row['id']),  # usa l'id reale
            'title': row['title'],
            'ingredients': row['ingredients'],
        }
        for _, row in df.iterrows()
    ]

    os.makedirs(index_path, exist_ok=True)

    indexer_fields = pt.IterDictIndexer(index_path)
    indexref = indexer_fields.index(
        documents_fields,
        fields=["title", "ingredients"],
        meta={'docno': 20, 'title': 512, 'ingredients': 1024}
    )

# Costruisci oggetto Index e motore di retrieval
index_fields = pt.IndexFactory.of(indexref)
bm25_ingredients = pt.terrier.Retriever(
    index_fields,
    wmodel="BM25",
    controls={"w": "1.0"},
    metadata=["docno", "ingredients"],
    field="ingredients"
)



✔️ Index already exists. Using existing index.
15:55:53.111 [main] WARN org.terrier.structures.FSADocumentIndex -- This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored


In [17]:
user_input = input("Enter a list of ingredients (comma-separated): ")

## RAG

In [18]:
result = bm25_ingredients.search(user_input)
print(result)

TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  6.82q/s]

    qid   docid    docno                                        ingredients  \
0     1  429472  2206467  ["1 12 cups long grain rice", "14 cup onion (D...   
1     1  546709  1771397  ["3 cups frozen tater tots", "3 tablespoons ke...   
2     1  163274  1792435  ["4 lb chicken breast", "1 Red pepper", "1 oni...   
3     1  152451  2137849  ["1 loaf Crusty Bread, Such As Ciabatta, Split...   
4     1  499422  1065708  ["4 large tomatoes", "3 egg yolks", "2 tablesp...   
..   ..     ...      ...                                                ...   
995   1  475415  1343911  ["1/2 skinless chicken breast", "1 cup baby po...   
996   1  499829  2116939  ["2 3-pound organic chickens", "2 lemons, quar...   
997   1  508243  1441934  ["1/3 cup whipped honey", "1/3 cup Dijon musta...   
998   1  516224   395406  ["2 c. lemon juice", "3 Tbsp. sugar", "1/2 c. ...   
999   1  527404  1641881  ["2 slices seeded bread", "1 mozzarella ball",...   

     rank      score               query  
0       

In [19]:
id = result.iloc[0]["docno"]
id = int(id)  # converti in intero
print(f"ID: {id}")

matched_row = df[df["id"] == id]
print(matched_row)


ID: 2206467
             id          title  \
429472  2206467  Mexicano Rice   

                                              ingredients  \
429472  ["1 12 cups long grain rice", "14 cup onion (D...   

                                               directions  \
429472  ['Add oil to pot and turn heat to medium/high....   

                                            link     source  \
429472  www.food.com/recipe/mexicano-rice-304280  Recipes1M   

                                                      NER  dir_word_count  \
429472  ["long grain rice", "onion", "water", "tomato ...              63   

        ner_len  
429472        6  


## One shot promts

In [20]:
prompt_system_ingrs = """
You are a creative and helpful cooking assistant.

The user will provide a list of available ingredients.  
Your task is to create a complete and realistic recipe using **only those ingredients** — do not invent or add anything not included in the list.

The recipe must include:
- A clear and descriptive title
- Step-by-step preparation instructions

Important rules:
- Do NOT use any ingredient that is not listed.
- If a necessary item (e.g., salt, oil, water) is missing, do NOT assume it's available — exclude it.
- You can adjust the form or quantity of the ingredients (e.g., chop, grate, blend), but not add new ones.
- If the ingredients are unusual together, invent a plausible or creative dish that still makes good use of them.

Output your response in the following JSON format:

{{
    "title": "recipe title",
    "list_of_steps": ["step 1", "step 2", ...]
}}

Example, take some ispiration from this:
{Example}

"""

In [21]:
response_schemas_2 = [
    ResponseSchema(
        name='title',
        description='The title of the recipe',
    ),
    ResponseSchema(
        name='list_of_steps',
        description='A list of steps to prepare the dish, only the list',
    )
]

In [22]:
output_parser_2 = StructuredOutputParser.from_response_schemas(response_schemas_2)

In [23]:
prompt_system_ingrs_ = ChatPromptTemplate.from_messages(
    [('system', prompt_system_ingrs), ('user', '{messages}')]
).partial(format_instructions=output_parser_2.get_format_instructions())

## LLM

In [24]:
llm = ChatOpenAI(model="gpt-4", temperature=0.7)
recipe_chain_2 = prompt_system_ingrs_ | llm | output_parser_2
response_2 = recipe_chain_2.invoke({"messages": user_input, "Example":matched_row})

In [25]:
print("Title:", response_2['title'])


print("\nPreparation Steps:")
for i, step in enumerate(response_2['list_of_steps'], 1):
    print(f"{i}. {step}")


Title: Simple Tomato Chicken

Preparation Steps:
1. 1. Clean the chicken and cut it into medium-sized pieces.
2. 2. Heat the oil in a pan over medium heat.
3. 3. Add the chicken pieces to the pan and cook until they turn golden brown.
4. 4. Cut the tomato into small pieces and add it to the pan.
5. 5. Cook until the tomato becomes soft and the chicken is fully cooked.
